In [2]:
import sqlite3
import pandas as pd

In [3]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [4]:
conn = sqlite3.connect('parch-and-posey.db')

In [5]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [6]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


In [7]:
pd.read_sql_query(sql='''
SELECT *
FROM orders;
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18
3,4,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24
4,5,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49
...,...,...,...,...,...,...,...,...,...,...,...
6907,6908,4501,2016-06-29 04:03:39,11,199,59,269,54.89,1490.51,479.08,2024.48
6908,6909,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
6909,6910,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74
6910,6911,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92


Task 1:

For each account, determine the average amount of each type of paper they purchased across their orders. Your result should have four columns - one for the account name and one for the average quantity purchased for each of the paper types for each account.

In [30]:
pd.read_sql_query(sql='''
SELECT name, AVG(standard_qty), AVG(gloss_qty), AVG(poster_qty)
FROM orders o
JOIN accounts a
ON a.id = o.account_id
GROUP BY name
ORDER BY name;
''', con=conn)

,name,AVG(standard_qty),AVG(gloss_qty),AVG(poster_qty)
0,3M,313.392857,279.750000,112.107143
1,ADP,354.633333,60.533333,61.983333
2,AECOM,363.777778,13.888889,16.666667
3,AES,254.666667,304.000000,98.333333
4,AIG,300.500000,349.000000,108.000000
...,...,...,...,...
345,World Fuel Services,208.000000,206.750000,15.000000
346,Xcel Energy,417.166667,157.000000,8.833333
347,Xerox,266.750000,19.000000,88.250000
348,Yum Brands,280.733333,38.666667,24.133333


Solution:

SELECT a.name, AVG(o.standard_qty) avg_stand, AVG(o.gloss_qty) avg_gloss, AVG(o.poster_qty) avg_post
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.name;

Task 2:

For each account, determine the average amount spent per order on each paper type. Your result should have four columns - one for the account name and one for the average amount spent on each paper type.

In [36]:
# -- WHERE name = 'Boeing' is remark

pd.read_sql_query(sql='''
SELECT name, AVG(standard_amt_usd), AVG(gloss_amt_usd), AVG(poster_amt_usd)
FROM orders o
JOIN accounts a
ON a.id = o.account_id
-- WHERE name = 'Boeing'
GROUP BY name
ORDER BY name;
''', con=conn)

,name,AVG(standard_amt_usd),AVG(gloss_amt_usd),AVG(poster_amt_usd)
0,3M,1563.830357,2095.327500,910.310000
1,ADP,1769.620333,453.394667,503.304667
2,AECOM,1815.251111,104.027778,135.333333
3,AES,1270.786667,2276.960000,798.466667
4,AIG,1499.495000,2614.010000,876.960000
...,...,...,...,...
345,World Fuel Services,1037.920000,1548.557500,121.800000
346,Xcel Energy,2081.661667,1175.930000,71.726667
347,Xerox,1331.082500,142.310000,716.590000
348,Yum Brands,1400.859333,289.613333,195.962667


Solution:

SELECT a.name, AVG(o.standard_amt_usd) avg_stand, AVG(o.gloss_amt_usd) avg_gloss, AVG(o.poster_amt_usd) avg_post
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.name;

Task 3:

Determine the number of times a particular channel was used in the web_events table for each sales rep. Your final table should have three columns - the name of the sales rep, the channel, and the number of occurrences. Order your table with the highest number of occurrences first.

In [37]:
pd.read_sql_query(sql='''
SELECT s.name, w.channel, COUNT (*) num_events
FROM web_events w
JOIN accounts a
ON a.id = w.account_id
JOIN sales_reps s
ON a.sales_rep_id = s.id
GROUP BY s.name, w.channel
ORDER BY s.name, num_events DESC;
''', con=conn)

,name,channel,num_events
0,Akilah Drinkard,direct,53
1,Akilah Drinkard,organic,15
2,Akilah Drinkard,facebook,9
3,Akilah Drinkard,banner,9
4,Akilah Drinkard,adwords,5
...,...,...,...
290,Vernita Plump,adwords,43
291,Vernita Plump,organic,40
292,Vernita Plump,facebook,36
293,Vernita Plump,twitter,16


Solution:

SELECT s.name, w.channel, COUNT(*) num_events
FROM accounts a
JOIN web_events w
ON a.id = w.account_id
JOIN sales_reps s
ON s.id = a.sales_rep_id
GROUP BY s.name, w.channel
ORDER BY num_events DESC;

Task 4:

Determine the number of times a particular channel was used in the web_events table for each region. Your final table should have three columns - the region name, the channel, and the number of occurrences. Order your table with the highest number of occurrences first.

In [38]:
pd.read_sql_query(sql='''
SELECT r.name, w.channel, COUNT (*) num_events
FROM web_events w
JOIN accounts a
ON a.id = w.account_id
JOIN sales_reps s
ON a.sales_rep_id = s.id
JOIN region r
ON s.region_id = r.id
GROUP BY r.name, w.channel
ORDER BY r.name, num_events DESC;
''', con=conn)

,name,channel,num_events
0,Midwest,direct,696
1,Midwest,facebook,125
2,Midwest,organic,117
3,Midwest,adwords,101
4,Midwest,twitter,71
5,Midwest,banner,59
6,Northeast,direct,1800
7,Northeast,facebook,335
8,Northeast,organic,317
9,Northeast,adwords,300


Solution:

SELECT r.name, w.channel, COUNT(*) num_events
FROM accounts a
JOIN web_events w
ON a.id = w.account_id
JOIN sales_reps s
ON s.id = a.sales_rep_id
JOIN region r
ON r.id = s.region_id
GROUP BY r.name, w.channel
ORDER BY num_events DESC;